In [14]:
import pandas as pd
import altair as alt
from vega_datasets import data
# Create a Dataframe from a CSV
dataCSV = pd.read_csv('yelp_academic_dataset_business.csv')

# Drop rows with any empty cells
dataCSV.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
dataCSV.head()

,latitude,postal_code,categories,longitude,city,name,state,address
0,40.017544,80302,"Gastropubs, Food, Beer Gardens, Restaurants, B...",-105.283348,Boulder,Oskar Blues Taproom,CO,921 Pearl St
1,45.588906,97218,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",-122.593331,Portland,Flying Elephants at PDX,OR,7000 NE Airport Way
2,45.511907,97214,"Antiques, Fashion, Used, Vintage & Consignment...",-122.613693,Portland,The Reclaimory,OR,4720 Hawthorne Ave
3,28.914482,32763,"Beauty & Spas, Hair Salons",-81.295979,Orange City,Great Clips,FL,2566 Enterprise Rd
4,33.747027,30316,"Gyms, Active Life, Interval Training Gyms, Fit...",-84.353424,Atlanta,Crossfit Terminus,GA,1046 Memorial Dr SE


In [15]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [16]:
RestData = dataCSV[dataCSV['categories'].str.contains('Restaurant')== True]
RestData.head()

,latitude,postal_code,categories,longitude,city,name,state,address
0,40.017544,80302,"Gastropubs, Food, Beer Gardens, Restaurants, B...",-105.283348,Boulder,Oskar Blues Taproom,CO,921 Pearl St
1,45.588906,97218,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",-122.593331,Portland,Flying Elephants at PDX,OR,7000 NE Airport Way
5,49.251342,V5V,"Restaurants, Thai",-123.101333,Vancouver,Bob Likes Thai Food,BC,3755 Main St
7,39.947007,43206,"Breakfast & Brunch, Restaurants",-82.997471,Columbus,Boxwood Biscuit,OH,740 S High St
12,42.541155,1960,"Food, Pizza, Restaurants",-70.973438,Peabody,Mr G's Pizza & Subs,MA,474 Lowell St


In [17]:
AsianRest = RestData[RestData['categories'].str.contains('Asian Fusion|Bangladeshi|Burmese|Cambodian|Cantonese|Chinese|Conveyor Belt Sushi|Dim Sum|Filipino|Guamanian|Hainan|Hawaiian|Himalayan/Nepalese|Hong Kong Style Cafe|Indian|Indonesian|Izakaya|Japanese|Japanese Curry|Korean|Laotian|Malaysian|Mongolian|Noodles|Pakistani|Pan Asia|Polynesian|Ramen|Shanghainese|Singaporean|Sri Lankan|Sushi Bars|SzechuanTaiwanese|Teppanyaki|Vietnamese|Bubble Tea|Hot Pot|Thai|Taiwanese|Szechuan|Pan Asian')== True]
AsianRest.head()

,latitude,postal_code,categories,longitude,city,name,state,address
5,49.251342,V5V,"Restaurants, Thai",-123.101333,Vancouver,Bob Likes Thai Food,BC,3755 Main St
41,30.234533,78735,"Bars, Nightlife, Cocktail Bars, Seafood, Resta...",-97.877262,Austin,Eurasia Sushi Bar & Seafood,TX,"7101 W Hwy 71, Ste C-13"
47,49.173723,V3V 4B9,"Indian, Restaurants",-122.889641,Surrey,Kerala Kitchen,BC,103- 9386 120 Street
53,28.800686,32771,"Restaurants, Chinese, Fast Food",-81.331712,Sanford,Panda Express,FL,1460 Rinehart Rd
75,42.468081,1907,"Restaurants, Chinese",-70.916752,Swampscott,Yan's China Bistro,MA,146 Humphrey St


In [18]:
restaurantCount = pd.read_csv('restaurantCountByState.csv')
restaurantCount.head()

,id,state,Number of Restaurant,url
0,1,Alabama,127,https://www.yelp.com/search?find_desc=Asian%20...
1,2,Alaska,112,https://www.yelp.com/search?find_desc=Asian%20...
2,4,Arizona,1100,https://www.yelp.com/search?find_desc=Asian%20...
3,5,Arkansas,81,https://www.yelp.com/search?find_desc=Asian%20...
4,6,California,8200,https://www.yelp.com/search?find_desc=Asian%20...


In [22]:
state_map = alt.topo_feature(data.us_10m.url, 'states')

click = alt.selection_multi(fields=['state'])

choropleth = (alt.Chart(state_map).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(restaurantCount, 'id', ['Number of Restaurant', 'state', 'url']))
.encode(
    color='Number of Restaurant:Q',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    tooltip=['state:N', 'Number of Restaurant:Q', 'url:N'])
.add_selection(click)
.project(type='albersUsa'))

bars = (
    alt.Chart(
        restaurantCount.nlargest(15, 'Number of Restaurant'),
        title='Top 15 States by Number of Restaurant').mark_bar().encode(
    x='Number of Restaurant',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    color='Number of Restaurant',
    y=alt.Y('state', sort='x'),
    tooltip = ['state:N', 'Number of Restaurant:Q', 'url:N'])
.add_selection(click))

# asianRestaurantsMap = alt.Chart(AsianRest).mark_circle().encode(
#     longitude='longitude:Q',
#     latitude='latitude:Q',
#     size=alt.value(20),
#     color=alt.value('#ff2400')
# #     tooltip=['name:N', 'address:N', 'city:N', 'state:N', 'categories:N']
# ).project(
#     "albersUsa"
# )

# restaurantMap = choropleth + asianRestaurantsMap
restaurantMap = choropleth | bars
restaurantMap

alt.HConcatChart(...)

In [23]:
restaurantMap.save("AsianRestaurantMap.html", embed_options={"render": "svg"})